### Friends canned laughter identification

In this notebook, we will try to use data from available friends
episodes to try to train a model using VGGish

In [ ]:
import sys
sys.path.append('../utils/')

In [ ]:
# local imports
import utils
import episode
# stdlib and package imports
import numpy as np
from matplotlib import pyplot as plt
# keras and ML imports
from keras.models import Sequential, model_from_yaml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize as sknormalize

#### Extracting data segments
Here we will extract labelled embeddings of wav data for each of the episodes

In [ ]:
# these are the episodes we have annotation data for
episodes = ['friends-s02-e{:0>2d}'.format(i) for i in range(1, 5)]

In [ ]:
# use VGGish to generate embeddings for each of the episode, and split data into chunks of 0.96s (no preserve_length)
X_raw, Y_raw, refs = episode.get_data(which_episodes=episodes, use_vggish=True, preserve_length=False)

In [ ]:
X_raw.shape, Y_raw.shape, refs.shape